# UF Research Computing  

![UF Research Computing Logo](images/ufrc_logo.png)


This tutorial is adapted from: [Understanding PyTorch with an example: a step-by-step tutorial](https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e) by Daniel Godoy.

In [30]:
# Re-establish the environment
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import random

# Function defined near end of 04_Optimizer_Loss.ipynb
from train_step import make_train_step 
lr = 1e-1
n_epochs = 1000
%run generate_data.py

# Reset the loss_fn, model and optimizer
loss_fn = nn.MSELoss(reduction='mean')
model = nn.Sequential(nn.Linear(1, 1)).to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)


## Dataset

In PyTorch, a **dataset** is represented by a regular **Python class** that inherits from the [**Dataset**](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class. You can think of it as a kind of a Python **list of tuples**, each tuple corresponding to **one point (features, label)**.

The most fundamental methods it needs to implement are:
* `__init__(self)`: it takes **whatever arguments** needed to build a **list of tuples** — it may be the name of a CSV file that will be loaded and processed; it may be **two tensors**, one for features, another one for labels; or anything else, depending on the task at hand.

  There is **no need to load the whole dataset in the constructor method* (`__init__`). If your **dataset is big** (tens of thousands of image files, for instance), loading it at once would not be memory efficient. It is recommended to **load them on demand** (whenever `__get_item__` is called).
  
* `__get_item__(self, index)`: it allows the dataset to be **indexed**, so it can work **like a list** (`dataset[i]`) — it must **return a tuple (features, label)** corresponding to the requested data point. We can either return the **corresponding slices** of our **pre-loaded** dataset or tensors or, as mentioned above, **load them on demand** (like in this [example](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)).

* `__len__(self)`: it should simply return the **size** of the whole dataset so, whenever it is sampled, its indexing is limited to the actual size.

Let’s build a simple custom dataset that takes two tensors as arguments: one for the features, one for the labels. For any given index, our dataset class will return the corresponding slice of each of those tensors. It should look like this:

In [31]:
from torch.utils.data import Dataset, TensorDataset

class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

# Wait, is this a CPU tensor now? Why? Where is .to(device)?
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))
(tensor([0.7713]), tensor([2.4745]))


Once again, you may be thinking *"why go through all this trouble to wrap a couple of tensors in a class?"*. And, once again, you do have a point… if a dataset is nothing else but a couple of tensors, we can use PyTorch’s [TensorDataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset) class, which will do pretty much what we did in our custom dataset above.

Did you notice we built our **training tensors** out of Numpy arrays but we **did not send them to a device**? So, they are **CPU** tensors now! **Why?**

We **don’t want our whole training data to be loaded into GPU tensors**, as we have been doing in our example so far, because **it takes up space** in our precious **graphics card’s RAM**.

OK, fine, but then again, **why** are we building a dataset anyway? We’re doing it because we want to use a…

## DataLoader

Until now, we have used the **whole training data** at every training step. It has been **batch gradient descent** all along. This is fine for our *ridiculously small dataset*, sure, but if we want to go serious about all this, we **must** use **mini-batch** gradient descent. Thus, we need mini-batches. Thus, we need to **slice** our dataset accordingly. Do you want to do it *manually*?! Me neither!

So we use PyTorch’s [**DataLoader**](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class for this job. We tell it which **dataset** to use (the one we just built in the previous section), the desired **mini-batch size** and if we’d like to **shuffle** it or not. That’s it!

Our **loader** will behave like an **iterator**, so we can **loop over it** and **fetch a different mini-batch** every time.

In [32]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

To retrieve a sample mini-batch, one can simply run the command below — it will return a list containing two tensors, one for the features, another one for the labels.

 ```next(iter(train_loader))```

How does this change our training loop? Let’s check it out!

In [33]:
losses = []
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        # the dataset "lives" in the CPU, so do our mini-batches
        # therefore, we need to send those mini-batches to the
        # device where the model "lives"
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9709]], device='cuda:0')), ('0.bias', tensor([1.0264], device='cuda:0'))])


Two things are different now: not only we have an *inner loop* to load each and every mini-batch from our `DataLoader` but, more importantly, we are now **sending only one mini-batch to the device**.

For *bigger datasets*, **loading data sample by sample** (into a **CPU** tensor) using **Dataset’s `__get_item__`** and then **sending all samples** that belong to the same **mini-batch at once to your GPU** (device) is the way to go in order to make the **best use of your graphics card’s RAM**.

Moreover, if you have **many GPUs** to train your model on, it is best to keep your dataset “agnostic” and assign the batches to different GPUs during training.

So far, we’ve focused on the **training data** only. We built a *dataset* and a *data loader* for it. We could do the same for the **validation** data, using the **split** we performed at the beginning of this post… or we could use `random_split` instead.

## Random Split

PyTorch’s [`random_split()`](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) method is an easy and familiar way of performing a **training-validation split**. Just keep in mind that, in our example, we need to apply it to the **whole dataset** (*not the training dataset we built in two sections ago*).

Then, for each subset of data, we build a corresponding `DataLoader`, so our code looks like this:

In [34]:
from torch.utils.data.dataset import random_split

x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

dataset = TensorDataset(x_tensor, y_tensor)

train_dataset, val_dataset = random_split(dataset, [80, 20])

train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

Now we have a **data loader** for our **validation set**, so, it makes sense to use it for the…

## Evaluation

This is the last part of our journey — we need to change the training loop to include the **evaluation of our model**, that is, computing the **validation loss**. The first step is to include another inner loop to handle the *mini-batches* that come from the *validation loader*, sending them to the same *device* as our model. Next, we make **predictions** using our model (line 23) and compute the corresponding **loss** (line 24).

That’s pretty much it, but there are **two small, yet important**, things to consider:

* [`torch.no_grad()`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.no_grad): even though it won’t make a difference in our simple model, it is a **good practice** to **wrap the validation** inner loop with this **context manager to disable any gradient calculation** that you may inadvertently trigger — **gradients belong in training**, not in validation steps;
* [`eval()`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.eval): the only thing it does is **setting the model to evaluation mode** (just like its `train()` counterpart did), so the model can adjust its behavior regarding some operations, like [**Dropout**](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout).

Now, our training loop should look like this:

In [35]:
losses = []
val_losses = []
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            model.eval()

            yhat = model(x_val)
            val_loss = loss_fn(y_val, yhat)
            val_losses.append(val_loss.item())

print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9343]], device='cuda:0')), ('0.bias', tensor([1.0281], device='cuda:0'))])


Is there **anything else** we can improve or change? Sure, there is **always something else** to add to your model — using a [**learning rate scheduler**](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate), for instance. But this post is already *waaaay too long*, so I will stop right here.

“Where is the full working code with all bells and whistles?”, you ask? You can find it [**here**](https://gist.github.com/dvgodoy/1d818d86a6a0dc6e7c07610835b46fe4), or as part of this repository, [**here**]()

## Final Thoughts

Although this post was *much longer* than I anticipated when I started writing it, I wouldn’t make it any different — I believe it has **most of the necessary steps** one needs go to trough in order to **learn**, in a **structured** and **incremental** way, how to **develop Deep Learning models using PyTorch**.

Hopefully, after finishing working through all code in this post, you’ll be able to better appreciate and more easily work your way through PyTorch’s official [tutorials](https://pytorch.org/tutorials/).